In [2]:
import pandas as pd
import sqlite3

In [8]:
import matplotlib.pyplot as plt

In [9]:
db_path = r'..\data\traffic_data_cleaned.sqlite'

In [10]:
conn = sqlite3.connect(db_path)

In [11]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:", tables)

Tables in the database:            name
0  traffic_data


In [12]:
table_name = 'traffic_data'

In [13]:
data = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5;", conn)
data.head()

,date,time,zone,race,sex,type,arrest_made,warning_issued,outcome,contraband_found,...,contraband_alcohol,contraband_other,frisk_performed,search_conducted,search_basis,reason_for_search,reason_for_stop,vehicle_make,vehicle_model,state
0,2014-01-01,01:05:00,000,white,male,vehicular,000,1,warning,000,...,000,000,000,000,000,000,000,000,000,New Hampshire
1,2014-01-01,13:05:00,000,000,male,vehicular,000,1,warning,000,...,000,000,000,000,000,000,000,000,000,New Hampshire
2,2014-01-01,13:09:00,000,000,female,vehicular,000,1,warning,000,...,000,000,000,000,000,000,000,000,000,New Hampshire
3,2014-01-01,01:11:00,000,white,male,vehicular,000,1,warning,000,...,000,000,000,000,000,000,000,000,000,New Hampshire
4,2014-01-01,01:14:00,000,000,male,vehicular,000,1,warning,000,...,000,000,000,000,000,000,000,000,000,New Hampshire


In [14]:
data.shape

(5, 22)

In [15]:
cursor = conn.cursor()

In [16]:
cursor.execute("SELECT COUNT(*) FROM traffic_data;")

In [17]:
num_rows = cursor.fetchone()[0]
num_rows

4487465

In [18]:
table_info = conn.execute("PRAGMA table_info(traffic_data);").fetchall()
columns = [col[1] for col in table_info]
dynamic_query = "SELECT " + ", ".join(
    [f"SUM(CASE WHEN {col} = '000' THEN 1 ELSE 0 END) AS {col}_count" for col in columns]
) + " FROM traffic_data;"


cursor.execute(dynamic_query)
column_000_counts = cursor.fetchone()


for col, count in zip(columns, column_000_counts):
    print(f"'{col}': {count} occurrences of '000'")


conn.close()

'date': 14 occurrences of '000'
'time': 3416252 occurrences of '000'
'zone': 3960586 occurrences of '000'
'race': 128456 occurrences of '000'
'sex': 50563 occurrences of '000'
'type': 0 occurrences of '000'
'arrest_made': 292092 occurrences of '000'
'warning_issued': 32265 occurrences of '000'
'outcome': 62292 occurrences of '000'
'contraband_found': 4405599 occurrences of '000'
'contraband_drugs': 4415747 occurrences of '000'
'contraband_weapons': 4419940 occurrences of '000'
'contraband_alcohol': 1070010 occurrences of '000'
'contraband_other': 4413973 occurrences of '000'
'frisk_performed': 3923163 occurrences of '000'
'search_conducted': 259822 occurrences of '000'
'search_basis': 4410301 occurrences of '000'
'reason_for_search': 4469703 occurrences of '000'
'reason_for_stop': 1950662 occurrences of '000'
'vehicle_make': 4169348 occurrences of '000'
'vehicle_model': 4257377 occurrences of '000'
'state': 0 occurrences of '000'


In [19]:
conn = sqlite3.connect(db_path)

In [20]:
table_info = conn.execute("PRAGMA table_info(traffic_data);").fetchall()
columns = [col[1] for col in table_info]

In [21]:
dynamic_query = "SELECT " + ", ".join(
    [f"SUM(CASE WHEN {col} = '000' THEN 1 ELSE 0 END) AS {col}_count" for col in columns]
) + " FROM traffic_data;"

In [22]:
cursor = conn.cursor()
cursor.execute(dynamic_query)
column_000_counts = cursor.fetchone()

In [23]:
conn.close()

In [27]:
plt.figure(figsize=(12, 6))
plt.bar(columns, column_000_counts, color='skyblue')
plt.xlabel('Columns', fontsize=12)
plt.ylabel('Count of Missing Values', fontsize=12)
plt.title('Count of Missing Values in Each Column', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
output_path = "column_none_counts.pdf"
plt.savefig(output_path, format="pdf")
plt.close()